# More detailed optimization example (using BOBYQA)

There are costs associated with waiting times. In addition, the operation of the system costs money depending on the time. Each successfully served client brings in a fixed profit. The number of orders accepted per day can be controlled.

[BOBYQA documentation](https://numericalalgorithmsgroup.github.io/pybobyqa/build/html/index.html)

## Importing modules

In [1]:
# Processing results arrays
import numpy as np

# Optiomizer
import pybobyqa

# Simulator
from queuesim.models import mmc_model, get_simulator_from_model

## Model parameters

In [2]:
# Arrivals to be simulated
count = 100_000

# Service process
mean_S = 80
c = 1

# Costs
profit_per_client = 100  # Revenue per customer served
cost_waiting = 0.02  # Costs per waiting second
operation_cost_per_second = 0.5

## Definition of the function to be minimized

In [3]:
def f(x):
    # Generate model
    global mean_S, c, count, profit_per_client, cost_waiting, operation_cost_per_second
    rho = x[0]
    mean_I = mean_S / rho / c
    model = mmc_model(mean_I, mean_S, c, count)
    
    # Simulation
    get_simulator_from_model(model).run()

    # Calculate auxiliary result variables
    waiting_time = model['Dispose'].statistic_client_waiting.mean
    clients_per_day = model['Source'].count / model['Process'].statistic_wip.time * 86400
    yield_per_client = profit_per_client - waiting_time * cost_waiting

    # Revenue
    return clients_per_day * yield_per_client - operation_cost_per_second * 86400

In [4]:
def minimize_f(x):
    return -f(x)

## Running the optimization

In [5]:
# Initial solution
x0 = np.array([0.75])

# Search range
a = np.array([0.5])
b = np.array([0.99])

# Running BOBYQA
soln = pybobyqa.solve(minimize_f, x0, bounds=(a, b))  # Actually corrent, but then the optimization runs endlessly (==">5Min"):  objfun_has_noise=True

## Optimization successful?

In [6]:
soln.flag == soln.EXIT_SUCCESS

True

In [7]:
print(soln)

****** Py-BOBYQA Results ******
Solution xmin = [0.87295696]
Objective value f(xmin) = -41605.75139
Needed 27 objective evaluations (at 27 points)
Approximate gradient = [-7.13237679e+10]
Approximate Hessian = [[7.71082964e+18]]
Exit flag = 0
Success: rho has reached rhoend
******************************



### Result (should be 88.8%)

In [8]:
print(round(soln.x[0]*100,1),"%")

87.3 %


## Verification per iteration via $[y-\epsilon, y+\epsilon]$

In [9]:
x=np.linspace(soln.x-0.02,soln.x+0.02,20)
y=[f([float(rho)]) for rho in x]  # Takes a little longer, quite simpile formulation without parallelization (approx. 1 min on fast machine)

In [12]:
print(round(x[np.argmax(y)][0]*100,1),"%")

86.3 %


Attention: $f(x)$ is not deterministic. If the iterative search returns a slightly different value, this does not mean that the optimization result is not the optimum.